In [4]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import  PromptTemplate
from langchain_openai import OpenAI
from dotenv import load_dotenv

load_dotenv('../../../.env')

llm = OpenAI()

relevant_collections_template = """You have access to a OGC API Features collection and should retrieve collection ID's that are relevant to a user query.

User Query: {user_query}
Returned from /collections endpoint: {collections_endpoint}

Based on the names and Norwegian descriptions, here are the possibly relevant datasets collections: 
"""

prompt = PromptTemplate(input_variables=['user_query', 'collections_endpoint'], template=relevant_collections_template)

relevant_collections_chain = LLMChain(llm=llm, prompt=prompt, output_key='relevant_collections')

In [13]:
from typing import Any
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Sequence
from langchain_openai import OpenAI, ChatOpenAI
import requests
from langchain.chains.openai_functions import create_structured_output_runnable
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.chains import APIChain


BASE_URL = 'http://localhost:9000'


def get_collection_information(collection_id: str): 
    res = requests.get(f'{BASE_URL}/collections/{collection_id}.json')
    return res.json()


class GetCollectionsInput(BaseModel):
    collection_ids: Sequence[str] = Field(..., description='List of IDs for relevant colletions')

class GetCollectionsTool(BaseTool):
    name = 'get_collections'
    args_schema = GetCollectionsInput
    description = 'Use this to get information about certain OGC API Features collections'

    def _run(self, collection_ids: Sequence[str], *args: Any, **kwargs: Any) -> Any:
        return [get_collection_information(coll_id) for coll_id in collection_ids]


llm = ChatOpenAI().bind(
    tools=[convert_to_openai_function(GetCollectionsTool)])

relevant_collections_template = """You have access to a OGC API Features collection and should get relevant properties from the collections listed to help answer the user's query.

User Query: {user_query}
Collections: {collections}

Based on the query and the properties of the listed collections, these are the relevant properties of each collection:
"""

prompt = PromptTemplate(input_variables=[
                        'user_query', 'collections_endpoint'], template=relevant_collections_template)
                        

chain = LLMChain(llm=llm, prompt=prompt)
chain.invoke({"user_query": "Harry was a chubby brown beagle who loved chicken",
             'collections': ['public.5001_25832_ar50_gml']})

ValueError: Value not declarable with JSON Schema, field: name='_callbacks_List[langchain_core.callbacks.base.BaseCallbackHandler]' type=BaseCallbackHandler required=True

In [3]:
llm = OpenAI()

summary_template = """Provide a summary of the below: 

Relevant collections: {relevant_collections}

Go!
"""

prompt2 = PromptTemplate(input_variables=['relevant_collections'], template=summary_template)
summary_chain = LLMChain(llm=llm, prompt=prompt2, output_key='summary')

In [5]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    # chains=[relevant_collections_chain, summary_chain],
    chains=[relevant_collections_chain,  summary_chain],
    input_variables=['user_query', 'collections_endpoint'],
    output_variables=['summary'],
    verbose=True
)

import requests

BASE_URL = 'http://localhost:9000'

collections_endpoint = requests.get(f'{BASE_URL}/collections').json()

overall_chain({
    'user_query': 'Gi meg alle veger med fartsgrense 100 km/t.',
    'collections_endpoint': collections_endpoint
    
})



> Entering new SequentialChain chain...

> Finished chain.


{'user_query': 'Gi meg alle veger med fartsgrense 100 km/t.',
 'collections_endpoint': {'links': [{'href': 'http://localhost:9000/collections',
    'rel': 'self',
    'type': 'application/json',
    'title': 'This document as JSON'},
   {'href': 'http://localhost:9000/collections.html',
    'rel': 'alternate',
    'type': 'text/html',
    'title': 'This document as HTML'}],
  'collections': [{'id': 'public.5001_25832_ar50_gml',
    'title': 'public.5001_25832_ar50_gml',
    'description': 'AR50 - Arealressurskart. Sentrale bruksområder er oversiktsplanlegging, konsekvensutredninger og utmarksforvaltning.',
    'extent': {'spatial': {'crs': 'http://www.opengis.net/def/crs/EPSG/0/4326',
      'bbox': [0, 0, 0, 0]}},
    'links': [{'href': 'http://localhost:9000/collections/public.5001_25832_ar50_gml',
      'rel': 'self',
      'type': 'application/json',
      'title': 'Metadata as JSON'},
     {'href': 'http://localhost:9000/collections/public.5001_25832_ar50_gml.html',
      'rel': 'a